# 1. 芯片初始化操作

In [10]:
import time
import json
import numpy as np
import matplotlib.pyplot as plt

from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *

from util import plot_v_cond

In [13]:
chip=CHIP(PS(host="192.168.1.10", port = 7, delay=0.3, debug=3),init=True)
# chip.set_device_cfg(deviceType=0)

Connected to 192.168.1.10:7
local ip: 192.168.1.15 local port: 51708
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: flt           	字节码: 09 00 00 0f ff
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 80
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 01 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_reset     	字节码: 0b 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_ss        	字节码: 0d 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ser_para_sel  	字节码: 0e 00 00 00 01

指令完整字节码:  55aa010900000fff
指令完整字节码:  55aa010100000080
指令完整字节码:  55aa010100000100
指令完整字节码:  55aa010b00000001
指令完整字节码:  55aa010d00000001
指令完整字节码:  55aa010e00000001
收到信息: 4 bb550000
收到信息: 4 bb550000
收到信息: 4 bb550000
收到信息: 4 bb550000
收到信息: 4 bb550000
收到信息: 4 bb550000
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: dac_in        	字节码: 02 00 02 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 01
模式: 1
	帧头:               	

In [3]:
chip.ps.debug = 3
# chip.set_op_mode2(read=True,row=True)
# chip.set_tia_gain(1)  
for i in range(256*16):
    print(f"第{i}次")
    voltage,cond = chip.adc.get_out2(num=4,dout_ram_start=0,read_voltage=0.1)

第0次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 04

指令完整字节码:  55aa0600000004
收到信息: 128 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
第1次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 04

指令完整字节码:  55aa0600000004
收到信息: 128 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
第2次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 04

指令完整字节码:  55aa0600000004
收到信息: 128 000000000000000000000000000000000000000000000000000000000000

ValueError: invalid literal for int() with base 16: ''

In [3]:
# chip.close()

# 2. 读器件

## 2.1 常规读操作

In [ ]:
chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.1,tg=5)
chip.set_tia_gain(3)
print(chip.get_setting_info())

In [ ]:
voltage,cond = chip.read(row_index=[0,1,2,3],col_index=[3,4,6,7],row_value=None,col_value=None,all_tia=False,check_tia=True)
print(f"电导:{cond}us\n电压:{voltage}v")

## 2.2 自定义读操作

In [ ]:
chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.3)
chip.set_tia_gain(1)
print(chip.get_setting_info())

In [ ]:

chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.3)
chip.set_tia_gain(1)

# 先需要reset所有的latch
chip.set_cim_reset()

# 配置行bank
chip.set_latch([0],row=True,value=None)
# chip.set_bank([i for i in range(8)],row=True,value=0xFFFF_FFFF)

# 配置列bank
# chip.set_latch([[2,4,1,3]],row=False,value=None)
chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)

# 产生读脉冲, 并读出tia值
chip.generate_read_pulse()
voltage,cond, = chip.get_tia_out([k for k in range(16)])

plot_v_cond(v= voltage,cond= cond)

# 3. 写器件

## 3.1 常规写操作

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(0.1)
chip.set_pulse_width(0.1)
print(chip.get_setting_info())

In [ ]:
chip.write_one(row_index=0,col_index=0)

## 3.2 自定义写操作

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(0.1)
chip.set_pulse_width(0.1)
print(chip.get_setting_info())

In [ ]:
# 先需要reset所有的latch
chip.set_cim_reset()

# 配置行bank
chip.set_latch([2,4,253,255],row=True,value=None)
chip.set_bank([i for i in range(8)],row=True,value=0xFFFF_FFFF)

# 配置列bank
chip.set_latch([[2,4,1,3]],row=False,value=None)
chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)

# 产生写配置
chip.generate_write_pulse()

# 测试

### 测试新版加速代码

In [7]:
# 行0,列53
# 行0,列66
# 行0,列79
# 行0,列92
# 行0,列103
# 行0,列168
# 行0,列181
# 行0,列194
# 行0,列231
chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)   
vres,cres = chip.read2(row_index=[0,1,2,3],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,check_tia=True,sum=False)

send_packets:接收超时!


In [3]:
chip.ps.debug = 3
# chip.set_op_mode2(read=True,row=True)
# chip.set_tia_gain(1)  
for i in range(256*16):
    print(f"第{i}次")
    voltage,cond = chip.adc.get_out2(num=16,dout_ram_start=0,read_voltage=0.1)

第0次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 01

指令完整字节码:  55aa0600000001
收到信息: 32 0000000000000000000000000000000000000000000000000000000000000000
第1次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 01

指令完整字节码:  55aa0600000001
收到信息: 32 0000000000000000000000000000000000000000000000000000000000000000
第2次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 01

指令完整字节码:  55aa0600000001
收到信息: 32 0000000000000000000000000000000000000000000000000000000000000000
第3次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 01

指令完整字节码:  55aa0600000001
收到信息: 32 0000000000000000000000000000000000000000000000000000000000000000
第4次
模式: 6
	帧头:               	字节码: 55 aa 06
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 01

指令完整字节码:  55aa0600000001
收到信息: 32 00000000000000000000000000000000000

ValueError: invalid literal for int() with base 16: ''

In [14]:
chip.ps.debug = 3

chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)                        # 第四步设置, TIA增益
for row in range(78):
    print(f"第{row}次读")
    vres,cres = chip.read2(row_index=[0],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,check_tia=True,sum=False)

模式: 4
	帧头:               	字节码: 55 aa 04
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 0c
	指令: pl_dac_v      	字节码: 03 00 00 00
	指令: pl_dac_v      	字节码: 03 01 00 00
	指令: pl_dac_v      	字节码: 03 02 00 00
	指令: pl_dac_v      	字节码: 03 03 00 00
	指令: pl_dac_v      	字节码: 03 04 00 00
	指令: pl_dac_v      	字节码: 03 05 00 00
	指令: pl_dac_v      	字节码: 03 06 00 00
	指令: pl_dac_v      	字节码: 03 07 00 00
	指令: pl_dac_v      	字节码: 03 08 00 00
	指令: pl_dac_v      	字节码: 03 09 00 00
	指令: pl_dac_v      	字节码: 03 0a 00 00
	指令: pl_dac_v      	字节码: 03 0b 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ins_num       	字节码: 1c 00 00 00 0c
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 80 00

指令完整字节码:  55aa040000000c03000000030100000302000003030000030400000305000003060000030700000308000003090000030a0000030b0000
指令完整字节码:  55aa011c0000000c
指令完整字节码:  55aa010100008000
收到信息: 4 bb550000
收到信息: 4 bb550000
收到信息: 4 cc550000
模式: 1
	帧头:     	字节码: 55 aa 01
	指令: gain	字节码: 10 00 00 00 01

指令完整字节

ValueError: invalid literal for int() with base 16: ''

In [ ]:
chip.ps.debug = False
vres,cres = chip.read2(row_index=[0,1,2,3,4],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,check_tia=True,sum=False)

for i in range(len(vres)):
    plot_v_cond(vres[i],cres[i],figsize=(16,4))

In [ ]:
chip.ps.debug = False
chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)
for row in range(256):
    print(f"第{row}行")
    vres,cres = chip.read2(row_index=[row],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,check_tia=True,sum=False)
    plot_v_cond(vres[0],cres[0],figsize=(16,4))


In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(0.1)
chip.set_pulse_width(0.1)
print(chip.get_setting_info())

In [ ]:
for i in range(50):
    ins_data = [
        CMD(PL_READ_ROW_PULSE,command_data=CmdData(0)),
    ]
    num = len(ins_data)
    ins_data.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(num)))
    ins_data.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

    pkts=Packet()
    pkts.append_single(ins_data,mode=4)
    pkts.append_single([CMD(INS_NUM,command_data=CmdData(num))],mode=1)
    pkts.append_single([CMD(FAST_COMMAND_1,command_data=CmdData(FAST_COMMAND1_CONF.cfg_ins_run))],mode=1)
    chip.ps.send_packets(pkts)

    vres,cres = chip.adc.get_out2(num=1,dout_ram_start=0,read_voltage=0.1)

    print(vres[0][10])
    plt.figure()
    plt.ylabel("v")
    plt.xlabel("TIA")
    plt.plot(vres[0], marker='o', linestyle='-', linewidth=2,label="1")
    plt.show()

## debug版本读

In [ ]:
row = 0
col = 194
res1=[]
res2=[]
res3=[]

chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)                        # 第四步设置, TIA增益
vres,cres = chip.read2(row_index=[row],col_index=[col],read_voltage=0.1,tg=5,check_tia=True,all_tia=False)
print(vres)
res1.append(vres[0])
for i in range(20):
    print("---------------------------------------reset---------------------------------------")
    # # --------------------------------------------------------------- reset
    chip.set_op_mode(read=False,row=False)
    chip.set_dac_write_V(0.9,tg=5)
    chip.set_tia_gain(1)
    chip.set_pulse_width(1)

    chip.write_one(row_index=row,col_index=col)
    time.sleep(1.2)

    chip.set_op_mode2(read=True,row=True)
    chip.set_tia_gain(1)                        # 第四步设置, TIA增益
    vres,cres = chip.read2(row_index=[row],col_index=[col],read_voltage=0.1,tg=5,check_tia=True,all_tia=False)
    print(vres)
    res2.append(vres[0])
    print("---------------------------------------set---------------------------------------")
    # --------------------------------------------------------------- forming
    chip.set_op_mode(read=False,row=True)
    chip.set_dac_write_V(5,tg=1+0.05*i)
    chip.set_tia_gain(1)
    chip.set_pulse_width(1)
    chip.write_one(row_index=row,col_index=col)
    time.sleep(1.2)

    # --------------------------------------------------------------- 读
    chip.set_op_mode2(read=True,row=True)
    chip.set_tia_gain(1)                        # 第四步设置, TIA增益
    vres,cres = chip.read2(row_index=[row],col_index=[col],read_voltage=0.1,tg=5,check_tia=True,all_tia=False)
    print(vres)
    res3.append(vres[0])

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.set_dac_write_V(5,tg=1.2)
chip.set_tia_gain(1)
chip.set_pulse_width(1)
chip.write_one(row_index=row,col_index=col)
time.sleep(1.2)


chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)                        # 第四步设置, TIA增益
vres,cres = chip.read2(row_index=[row],col_index=[col],read_voltage=0.1,tg=5,check_tia=True,all_tia=False)
print(vres)
res3.append(vres[0])

In [ ]:
plt.figure()
plt.ylabel("v")
plt.xlabel("TIA")
plt.plot(res2, marker='o', linestyle='-', linewidth=2,label="reset")
plt.plot(res3, marker='o', linestyle='-', linewidth=2,label="formig")
plt.legend()
plt.show()

# plt.figure()
# plt.ylabel("v")
# plt.xlabel("TIA")
# plt.plot(res3, marker='o', linestyle='-', linewidth=2,label="formig")
# plt.show()

In [ ]:
chip.set_op_mode(read=False,row=False)
chip.set_dac_write_V(5,tg=5)
chip.set_tia_gain(1)
print(chip.get_setting_info())

In [ ]:
chip.set_op_mode2(read=True,row=True)
chip.set_tia_gain(1)                        # 第四步设置, TIA增益
vres,cres = chip.read2(row_index=[0],col_index=[66],read_voltage=0.1,tg=5,check_tia=True,all_tia=False)
print(vres)

In [ ]:
chip.set_op_mode(read=False,row=False)
chip.set_pulse_width(1)
chip.write2(row_index=[0],col_index=[66],write_voltage=5,tg=5)


In [ ]:
voltage,cond = chip.get_tia_out([k for k in range(16)])
print(voltage)
plot_v_cond(voltage,cond)


In [ ]:
# chip.set_cim_reset()
# chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)
# chip.set_bank([i for i in range(8)],row=True,value=0xFFFF_FFFF)
for i in range(10):
    chip.generate_read_pulse()

    voltage,cond = chip.get_tia_out([k for k in range(16)])
    print(voltage)
    plot_v_cond(voltage,cond)

In [ ]:
for i in range(10):
    # chip.generate_read_pulse()
    voltage,cond = chip.get_tia_out([k for k in range(16)])
    print(voltage)
    plot_v_cond(voltage,cond)

In [ ]:
for i in range(20):
    # 先需要reset所有的latch
    chip.set_cim_reset()

    # 配置行bank
    chip.set_latch([0],row=True,value=None)
    chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)
    # chip.set_latch([i for i in range(256)],row=False,value=None)

    # 配置列bank
    # chip.set_latch([[2,4,1,3]],row=False,value=None)
    # chip.set_bank([i for i in range(8)],row=False,value=0xFFFF_FFFF)

    # 产生读脉冲, 并读出tia值
    chip.generate_read_pulse()
    voltage,cond, = chip.get_tia_out([k for k in range(16)])

    print(voltage)
    plot_v_cond(voltage,cond)
    if voltage[10]<0.44 or voltage[10]>0.45:
        break

In [ ]:
for i in range(20):
    voltage,cond = chip.read(row_index=[0],col_index=[i for i in range(256)],row_value=None,col_value=None,all_tia=True,check_tia=False)
    plot_v_cond(voltage,cond)

In [37]:
op_bank_cfg_done_delay_cyc = 1000
op_cim_data_cfg_done_delay_cyc = 1000
reg_clk_done_delay_cyc = 1000
latch_clk_done_delay_cyc = 1000
# op_adc_avrg_done_delay_cyc = 1000
# op_dac_cfg_done_delay_cyc = 1000
# op_row_pulse_done_delay_cyc = 1000
# op_col_pulse_done_delay_cyc = 1000
# op_cim_rstn_done_delay_cyc = 1000

pkts=Packet()
pkts.append_cmdlist([
    # CMD(OP_BANK_CFG_DONE_DELAY_CYC,command_data=CmdData(1000)),
    CMD(OP_CIM_DATA_CFG_DONE_DELAY_CYC,command_data=CmdData(10000)),
    CMD(REG_CLK_DONE_DELAY_CYC,command_data=CmdData(10000)),
    CMD(LATCH_CLK_DONE_DELAY_CYC,command_data=CmdData(10000)),
    # CMD(OP_ADC_AVRG_DONE_DELAY_CYC,command_data=CmdData(1000)),
    # CMD(OP_DAC_AVRG_DONE_DELAY_CYC,command_data=CmdData(1000)),
    # CMD(OP_ROW_PULSE_DONE_DELAY_CYC,command_data=CmdData(1000)),
    # CMD(OP_COL_PULSE_DONE_DELAY_CYC,command_data=CmdData(1000)),
    # CMD(OP_CIM_RSTN_DONE_DELAY_CYC,command_data=CmdData(1000)),
],mode=1)
chip.ps.send_packets(pkts)

## debug读一行

In [59]:
chip.set_op_mode(read=True,row=True)
chip.set_dac_read_V(0.1,tg=5)
chip.set_tia_gain(1)

In [ ]:
chip.set_cim_reset()
chip.generate_read_pulse()
print(chip.get_tia_out([i for i in range(16)])[0])

In [ ]:
# 行0,列53
# 行0,列66
# 行0,列79
# 行0,列92
# 行0,列103
# 行0,列168
# 行0,列181
# 行0,列194
# 行0,列231

In [ ]:
chip.set_dac_read_V(0.1,tg=5)
voltage,cond = chip.read(row_index=[0],col_index=[i for i in range(256)],row_value=None,col_value=None,check_tia=False)
plot_v_cond(voltage[0],cond[0])


chip.set_dac_read_V(0.5,tg=5)
voltage,cond = chip.read(row_index=[0],col_index=[i for i in range(256)],row_value=None,col_value=None,check_tia=False)
plot_v_cond(voltage[0],cond[0])
# chip.set_dac_read_V(0.3,tg=5)
# voltage,cond = chip.read(row_index=[0],col_index=[i for i in range(256)],row_value=None,col_value=None,check_tia=True)
# plot_v_cond(voltage,cond)